Zadanie: kanały sprzedaży
Do tej pory stworzyliśmy dwie zakładki dashboardu, które dotyczą ogólnego poziomu sprzedaży oraz analityki dotyczącej produktów. Twoim zadaniem będzie stworzenie trzeciej zakładki, która zawierać będzie dane dotyczące kanałów sprzedaży ('Store_type').
Zarząd powinien dostać informacje przynajmniej na temat tego, w jakich dniach tygodnia dokonuje się najwięcej sprzedaży w zależności od kanału sprzedaży.
Zarząd chciałby również dowiedzieć się czegoś więcej o klientach każdego z kanałów sprzedaży. Sposób prezentacji oraz funkcjonalność widgetów pozostawiamy Tobie.
Życzymy powodzenia, do dzieła!

ZADANIE W ODDZIELNYCH PLIKACH app.py tab.py

In [29]:
import pandas as pd
import datetime as dt
import os
import dash as dash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go

Tworzenie klasy dla danych wejściowych

In [21]:
class db:
    def __init__(self):
        self.transactions = db.transation_init()
        self.cc = pd.read_csv(r'db\country_codes.csv',index_col=0)
        self.customers = pd.read_csv(r'db\customers.csv',index_col=0)
        self.prod_info = pd.read_csv(r'db\prod_cat_info.csv')

Przekształcenie danych do DataFrame i do formatu dat

In [22]:
@staticmethod
def transation_init():
    transactions = pd.DataFrame()
    src = r'db\transactions'
    for filename in os.listdir(src):
        transactions = transactions.append(pd.read_csv(os.path.join(src,filename),index_col=0))

    def convert_dates(x):
        try:
            return dt.datetime.strptime(x,'%d-%m-%Y')
        except:
            return dt.datetime.strptime(x,'%d/%m/%Y')

    transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))

    return transactions

Połączenie baz danych

In [23]:
def merge(self):
    df = self.transactions.join(self.prod_info.drop_duplicates(subset=['prod_cat_code'])
    .set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')

    df = df.join(self.prod_info.drop_duplicates(subset=['prod_sub_cat_code'])
    .set_index('prod_sub_cat_code')['prod_subcat'],on='prod_subcat_code',how='left')

    df = df.join(self.customers.join(self.cc,on='country_code')
    .set_index('customer_Id'),on='cust_id')

    self.merged = df


    df = db()
    df.merge()

Tworzenie layoutu dashboardu

In [24]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
app.layout = html.Div([html.Div([dcc.Tabs(id='tabs',value='tab-1',children=[
    dcc.Tab(label='Sprzedaż globalna',value='tab-1'),
    dcc.Tab(label='Produkty',value='tab-2')]),
    html.Div(id='tabs-content')],
    
    style={'width':'80%','margin':'auto'})],
    style={'height':'100%'})



[Tabs(children=[Tab(label='Sprzedaż globalna', value='tab-1'), Tab(label='Produkty', value='tab-2')], id='tabs', value='tab-1'),
 Div(id='tabs-content')]

Ustawienie zakłądki sprzedaż globalna

In [27]:
def render_tab(df):

    layout = html.Div([html.H1('Sprzedaż globalna',style={'text-align':'center'}),
                        html.Div([dcc.DatePickerRange(id='sales-range',
                        start_date=df['tran_date'].min(),
                        end_date=df['tran_date'].max(),
                        display_format='YYYY-MM-DD')],style={'width':'100%','text-align':'center'}),
                        html.Div([html.Div([dcc.Graph(id='bar-sales')],style={'width':'50%'}),
                        html.Div([dcc.Graph(id='choropleth-sales')],style={'width':'50%'})],style={'display':'flex'})
                        ])

    return layout

Ustawienie zakładki z produktami

In [28]:
def render_tab(df):

    grouped = df[df['total_amt']>0].groupby('prod_cat')['total_amt'].sum()
    fig = go.Figure(data=[go.Pie(labels=grouped.index,values=grouped.values)],layout=go.Layout(title='Udział grup produktów w sprzedaży'))

    layout = html.Div([html.H1('Produkty',style={'text-align':'center'}),

                        html.Div([html.Div([dcc.Graph(id='pie-prod-cat',figure=fig)],style={'width':'50%'}),
                        html.Div([dcc.Dropdown(id='prod_dropdown',
                                    options=[{'label':prod_cat,'value':prod_cat} for prod_cat in df['prod_cat'].unique()],
                                    value=df['prod_cat'].unique()[0]),
                                    dcc.Graph(id='barh-prod-subcat')],style={'width':'50%'})],style={'display':'flex'}),
                                    html.Div(id='temp-out')
                        ])

    return layout